In [5]:
import RLMS_HSE.AIDS as aids
import RLMS_HSE.reading as readrlms
# ===================================
from importlib import reload  

Что надо?
1. Создание модели
2. Генерация такого дф агрегированных цен, который не истощает выборку панели
3. Реализация кроссекционной SUR-МОДЕЛИ
4. Разработка функция для получения параметров SUR-оценки
5. Тесты на SUR-коэффициенты
6. "Красивый вывод" результатов SUR-AIDS (тепловые карты и проч.)
7. Включение неучтеных товаров агрегации в агрегаты

In [ ]:
readrlms = reload(readrlms)
aids = reload(aids)

In [2]:
readrlms.FAST_variable_hh()

Волны 1993 года не существует.
Загружен 1994
Загружен 1995
Загружен 1996
Волны 1997 года не существует.
Загружен 1998
Волны 1999 года не существует.
Загружен 2000
Загружен 2001
Загружен 2002
Загружен 2003
Загружен 2004
Загружен 2005
Загружен 2006
Загружен 2007
Загружен 2008
Загружен 2009
Загружен 2010
Загружен 2011
Загружен 2012
Загружен 2013
Загружен 2014
Загружен 2015
Загружен 2016
Загружен 2017
Загружен 2018
Загружен 2019
Загружен 2020
Загружен 2021


In [9]:
aids.FAST_AIDS_matrices(readrlms.FAST_HH_DFS)

Матрица 1994 построена
Матрица 1995 построена
Матрица 1996 построена
Матрица 1998 построена
Матрица 2000 построена
Матрица 2001 построена
Матрица 2002 построена
Матрица 2003 построена
Матрица 2004 построена
Матрица 2005 построена
Матрица 2006 построена
Матрица 2007 построена
Матрица 2008 построена
Матрица 2009 построена
Матрица 2010 построена
Матрица 2011 построена
Матрица 2012 построена
Матрица 2013 построена
Матрица 2014 построена
Матрица 2015 построена
Матрица 2016 построена
Матрица 2017 построена
Матрица 2018 построена
Матрица 2019 построена
Матрица 2020 построена
Матрица 2021 построена


## Реализация рабочих функций генерации коэффициентов 

In [10]:
def alpha_AIDS(df, price_index='Stone', style='Bondarev', constrained=False,  homo=True, sym=True, fitted=False):
    df=aids.SUR_AIDS(df, price_index=price_index, style=style, constrained=constrained,  homo=homo, sym=sym, fitted=fitted).fit().params
    fin={}
    for par in df.index:
        if '_const' in par:
            fin[par]=df[par]
    fin['Неалкогольные напитки_const']=1-sum(fin.values())
    return pd.Series(fin)

In [11]:
def beta_AIDS(df, price_index='Stone', style='Bondarev', constrained=False,  homo=True, sym=True, fitted=False):
    df=aids.SUR_AIDS(df, price_index=price_index, style=style, constrained=constrained,  homo=homo, sym=sym, fitted=fitted).fit().params
    fin={}
    for par in df.index:
        if '_Остаток' in par:
            fin[par]=df[par]
    fin['Неалкогольные напитки_const']=-sum(fin.values())
    return pd.Series(fin)

In [12]:
def gamma_AIDS(df, price_index='Stone', style='Bondarev', constrained=False,  homo=True, sym=True, fitted=False):
    f_gamma=aids.SUR_AIDS(df, price_index=price_index, style=style, constrained=constrained,  homo=homo, sym=sym, fitted=fitted).fit().params
    f_list=[]
    for par in list(f_gamma.index):
        if '_Цена' in par:
            f_list.append(par)
    f_gamma=f_gamma.loc[f_list]
    
    for i in ['_Цена на Бакалея','_Цена на Овощи-фрукты',
              '_Цена на Мясо-рыба','_Цена на Молоко',
              '_Цена на Кондитерские изделия','_Цена на Алкоголь',
              '_Цена на Неалкогольные напитки']:
        f_gamma_2=[]
        for j in f_gamma.index:
            if i in j:
                f_gamma_2.append(j)
        f_gamma['Неалкогольные напитки{0}'.format(i)]=-f_gamma.loc[f_gamma_2].sum()
    return f_gamma